# Basis and reconstruction of 64 pixel image.
stough 202-
DIP

Decompose and partially reconstruct according to the Haar
basis.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

from wavelet_utils import (make_haar_matrix,
                           make_random_basis,
                           make_klt_basis,
                           make_dct_matrix,
                           make_standard_matrix)

In [ ]:
print('haarReconstructionDemo: Works only on 64 pixel images...')

I = plt.imread('../dip_pics/surprise.png')
# I = plt.imread('strawberry.png')


H = make_haar_matrix(8)

#The transform image, an image of coefficients wrt the Haar basis.
CT = np.zeros(I.shape)
for chan in range(3):
    CT[...,chan] = np.matmul(H, np.matmul(I[...,chan], H.transpose()))


#RI will represent the reconstructed image as we add back more
#Haar patterns
RI = np.zeros(I.shape) #Still should be 8x8


#Visual
fh, axh = plt.subplots(8,8, figsize=(8,8), num='Haar Basis Images and Coefficients')
fr, axr = plt.subplots(8,8, figsize=(8,8), num='Partial Reconstructions from Large Scale to Small')


# We're going to reconstruct according to distance from the
# 0,0 (the first Haar basis, average calculator). Notice,
# this order is independent of the actual image data.
xs = np.meshgrid(np.arange(8), np.arange(8), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)
dists = np.sum(coords*coords, axis=1)
darg = np.argsort(dists) #sorts in increasing order
is_mags = False

#If we were to use magnitude of the coefficient...
# mags = CT[...,0].ravel(order='F')
# darg = np.argsort(np.abs(mags))
# darg = list(reversed(darg))
# is_mags = True

#
#c = 1 #used to display the order of reconstruction.

for which_axis, which_basis in enumerate(darg):
    i,j = coords[which_basis] #coords[darg[x]] if x in range(len(darg))
    
    wa_i = i
    wa_j = j
    
    if is_mags:
        wa_i = which_axis//8
        wa_j = which_axis%8

    #Construct that Haar basis and display it
    Bij = np.outer(H[i, :], H[j, :])
    axh[wa_i][wa_j].imshow(Bij, cmap='gray', vmin=-.5, vmax=.5)
    axh[wa_i][wa_j].axes.get_xaxis().set_visible(False)
    axh[wa_i][wa_j].axes.get_yaxis().set_visible(False)
    # axh[i][j].set_title('c_%d_%d: %6.3f' % (i, j, T[i,j]))
    # https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.text
    axh[wa_i][wa_j].text(0, 6, r'$\pi:%6.3f$' % CT[i, j, :].sum(), fontsize=6, color='cyan')

    # Add the amount of that basis that was in the original image to
    # the running total, or reconstruction.
    # RI = RI + T[i, j] * Bij #2D
    for chan in range(3):
        RI[...,chan] += CT[i,j,chan]*Bij

    RIcopy = RI.copy().clip(0,1)
    axr[wa_i][wa_j].imshow(RIcopy)
    axr[wa_i][wa_j].axes.get_xaxis().set_visible(False)
    axr[wa_i][wa_j].axes.get_yaxis().set_visible(False)
    # axr[i][j].text(0, 6, '%d' % c, fontsize=10, color='cyan')
    #The order of reconstruction isn't that informative, but useful for debugging.
    #c += 1


plt.show()

# fh.canvas.set_window_title('Haar Basis Images and Coefficients')
fh.tight_layout()

# fr.canvas.set_window_title('Partial Reconstructions from Large Scale to Small')
fr.tight_layout()